# TIPM Demo: US-China Electronics Tariff Analysis

This notebook demonstrates the Tariff Impact Propagation Model (TIPM) by analyzing the impact of a hypothetical 25% tariff on Chinese electronics imports to the United States.

## Scenario Overview
- **Tariff Rate**: 25%
- **Affected Products**: Electronics (HS codes 8517, 8525, 8528)
- **Origin**: China (CN)
- **Destination**: United States (US)
- **Analysis Scope**: 6-layer impact propagation through global economy

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Import TIPM modules
from tipm.core import TIPMModel, TariffShock
from tipm.config import TIPMConfig
from tipm.utils.data_utils import DataLoader
from tipm.utils.visualization_utils import TIPMVisualizer

# Set up plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("TIPM libraries imported successfully!")

## 1. Initialize TIPM Model

First, we'll set up the TIPM model with default configuration optimized for US-China trade analysis.

In [ ]:
# Initialize TIPM configuration
config = TIPMConfig(
    random_seed=42,
    confidence_threshold=0.7,
    max_prediction_horizon=12  # 12 months
)

# Create TIPM model instance
model = TIPMModel(config)

print(f"TIPM Model initialized with {len(model.config.__dict__)} configuration parameters")
print(f"Model version: {config.model_version}")

## 2. Generate Sample Training Data

For this demo, we'll generate synthetic training data. In a production environment, this would be replaced with historical trade, policy, and economic data.

In [ ]:
# Initialize data loader
data_loader = DataLoader()

# Generate sample trade data
countries = ['US', 'CN', 'DE', 'JP', 'SG', 'KR', 'MX', 'CA']
years = [2020, 2021, 2022, 2023]

print("Generating sample trade data...")
trade_data = data_loader.load_trade_data(countries, years)

print(f"Generated {len(trade_data):,} trade records")
print(f"Total trade value: ${trade_data['trade_value'].sum():,.0f}")

# Display sample of trade data
trade_data.head()

In [ ]:
# Generate sample policy data
from datetime import datetime, timedelta

start_date = datetime(2020, 1, 1)
end_date = datetime(2023, 12, 31)

print("Generating sample policy data...")
policy_data = data_loader.load_policy_data((start_date, end_date))

print(f"Generated {len(policy_data)} policy announcements")

# Display sample policies
policy_data.head()

## 3. Train TIPM Model

Now we'll train the 6-layer TIPM model using our sample data.

In [ ]:
# Prepare training data for all layers
training_data = {
    'tariff_shocks': policy_data,
    'trade_flows': trade_data,
    'industry_responses': None,  # Would include industry survey data
    'firm_responses': None,      # Would include firm-level response data
    'consumer_impacts': None,    # Would include price and CPI data
    'geopolitical_events': None  # Would include social media and news data
}

print("Training TIPM model...")
print("This may take a few minutes...")

# Train the model
model.fit(training_data)

print("✅ Model training completed!")
print(f"Model status: {'Trained' if model.is_trained else 'Not trained'}")

## 4. Define Tariff Shock Scenario

Let's create our tariff shock scenario: 25% tariff on Chinese electronics.

In [ ]:
# Create tariff shock scenario
tariff_shock = TariffShock(
    tariff_id="US_CHINA_ELECTRONICS_2024_001",
    hs_codes=["8517", "8525", "8528"],  # Telecommunications, cameras, monitors
    rate_change=0.25,  # 25% tariff
    origin_country="CN",
    destination_country="US",
    effective_date="2024-08-01",
    policy_text="""The United States Trade Representative announces a 25% tariff on imports of 
    telecommunications equipment, digital cameras, and computer monitors from China, 
    effective August 1, 2024. This measure responds to ongoing trade disputes and aims 
    to protect domestic electronics manufacturing."""
)

print("Tariff Shock Scenario Created:")
print(f"  ID: {tariff_shock.tariff_id}")
print(f"  Rate: {tariff_shock.rate_change * 100}%")
print(f"  Route: {tariff_shock.origin_country} → {tariff_shock.destination_country}")
print(f"  Products: {', '.join(tariff_shock.hs_codes)}")
print(f"  Effective: {tariff_shock.effective_date}")

## 5. Run Impact Prediction

Now we'll run the tariff impact prediction through all 6 layers of the TIPM model.

In [ ]:
print("Running tariff impact prediction...")
print("Processing through 6 model layers:")

# Run prediction
prediction = model.predict(tariff_shock)

print("\n✅ Prediction completed!")
print("\nModel Confidence Scores:")
for layer, confidence in prediction.confidence_scores.items():
    status = "🟢" if confidence > 0.8 else "🟡" if confidence > 0.6 else "🔴"
    print(f"  {status} {layer}: {confidence:.1%}")

## 6. Analyze Results by Layer

Let's examine the prediction results from each layer of the TIPM model.

In [ ]:
# Layer 2: Trade Flow Impact
print("=== LAYER 2: TRADE FLOW IMPACT ===")
print(f"Affected trade routes: {len(prediction.trade_flow_impact.affected_routes)}")

if prediction.trade_flow_impact.supply_chain_disruption:
    print("\nSupply Chain Disruption by Sector:")
    for sector, disruption in list(prediction.trade_flow_impact.supply_chain_disruption.items())[:5]:
        impact_level = "High" if disruption > 0.7 else "Medium" if disruption > 0.4 else "Low"
        print(f"  📊 {sector}: {disruption:.1%} ({impact_level})")

if prediction.trade_flow_impact.alternative_sources:
    print("\nAlternative Supply Sources Identified:")
    for route, alternatives in list(prediction.trade_flow_impact.alternative_sources.items())[:3]:
        print(f"  🔄 {route}: {', '.join(alternatives[:3])}")

In [ ]:
# Layer 3: Industry Response
print("=== LAYER 3: INDUSTRY RESPONSE ===")

if hasattr(prediction.industry_response, 'sector_impacts'):
    print("\nSector Impact Analysis:")
    for sector, impact in list(prediction.industry_response.sector_impacts.items())[:5]:
        print(f"  🏭 {sector}: {impact:.1%} impact")

if hasattr(prediction.industry_response, 'employment_effects'):
    print("\nEmployment Effects:")
    for sector, effect in list(prediction.industry_response.employment_effects.items())[:5]:
        direction = "📈" if effect > 0 else "📉"
        print(f"  {direction} {sector}: {effect:.1%} employment change")

In [ ]:
# Layer 4: Firm Impact
print("=== LAYER 4: FIRM-LEVEL IMPACT ===")

if hasattr(prediction.firm_impact, 'layoff_risk'):
    print("\nLayoff Risk by Sector:")
    for sector, risk in list(prediction.firm_impact.layoff_risk.items())[:5]:
        risk_level = "High" if risk > 0.7 else "Medium" if risk > 0.4 else "Low"
        print(f"  ⚠️  {sector}: {risk:.1%} ({risk_level} risk)")

if hasattr(prediction.firm_impact, 'adaptation_strategies'):
    print("\nFirm Adaptation Strategies:")
    strategy_counts = {}
    for sector, strategy in prediction.firm_impact.adaptation_strategies.items():
        strategy_counts[strategy] = strategy_counts.get(strategy, 0) + 1
    
    for strategy, count in strategy_counts.items():
        print(f"  📋 {strategy}: {count} sectors")

In [ ]:
# Layer 5: Consumer Impact
print("=== LAYER 5: CONSUMER IMPACT ===")

if hasattr(prediction.consumer_impact, 'price_increases'):
    print("\nPrice Increases by Sector:")
    for sector, increase in list(prediction.consumer_impact.price_increases.items())[:5]:
        print(f"  💰 {sector}: +{increase:.1%} price increase")

if hasattr(prediction.consumer_impact, 'welfare_effects'):
    total_welfare_loss = sum(prediction.consumer_impact.welfare_effects.values())
    print(f"\nEstimated Consumer Welfare Impact: {total_welfare_loss:.1%}")

In [ ]:
# Layer 6: Geopolitical Impact
print("=== LAYER 6: GEOPOLITICAL IMPACT ===")

if hasattr(prediction.geopolitical_impact, 'social_tension'):
    avg_tension = np.mean(list(prediction.geopolitical_impact.social_tension.values()))
    tension_level = "High" if avg_tension > 0.7 else "Medium" if avg_tension > 0.4 else "Low"
    print(f"\nAverage Social Tension Level: {avg_tension:.1%} ({tension_level})")

if hasattr(prediction.geopolitical_impact, 'policy_responses'):
    print("\nPredicted Policy Responses:")
    response_counts = {}
    for sector, response in prediction.geopolitical_impact.policy_responses.items():
        response_counts[response] = response_counts.get(response, 0) + 1
    
    for response, count in response_counts.items():
        print(f"  🏛️  {response}: {count} sectors")

## 7. Create Visualizations

Let's create comprehensive visualizations of our tariff impact analysis.

In [ ]:
# Initialize visualizer
visualizer = TIPMVisualizer()

# Create comprehensive impact summary
print("Creating impact visualization dashboard...")
visualizer.plot_impact_summary(prediction)

print("📊 Visualization completed!")

## 8. Scenario Analysis

Let's run a multi-scenario analysis to compare different tariff rates.

In [ ]:
# Create multiple tariff scenarios
scenarios = []
tariff_rates = [0.10, 0.15, 0.25, 0.35]  # 10%, 15%, 25%, 35%

print("Running multi-scenario analysis...")

scenario_results = []
for rate in tariff_rates:
    scenario_shock = TariffShock(
        tariff_id=f"SCENARIO_{int(rate*100)}PCT",
        hs_codes=["8517", "8525", "8528"],
        rate_change=rate,
        origin_country="CN",
        destination_country="US",
        effective_date="2024-08-01",
        policy_text=f"{int(rate*100)}% tariff scenario"
    )
    
    scenario_prediction = model.predict(scenario_shock)
    scenario_results.append({
        'tariff_rate': rate,
        'confidence': scenario_prediction.confidence_scores.get('overall_confidence', 0),
        'prediction': scenario_prediction
    })
    
    print(f"  ✅ {int(rate*100)}% tariff scenario completed")

print("\n📊 Scenario Analysis Summary:")
for result in scenario_results:
    rate_pct = int(result['tariff_rate'] * 100)
    confidence = result['confidence']
    print(f"  {rate_pct}% Tariff - Confidence: {confidence:.1%}")

## 9. Key Insights and Conclusions

Based on our TIPM analysis, here are the key insights from the US-China electronics tariff scenario:

In [ ]:
print("=== KEY INSIGHTS FROM TIPM ANALYSIS ===")
print()

# Extract key metrics
overall_confidence = prediction.confidence_scores.get('overall_confidence', 0)

print(f"📈 PREDICTION CONFIDENCE: {overall_confidence:.1%}")
print()

print("🔍 IMPACT PROPAGATION SUMMARY:")
print("  1. Trade Flow Layer: Supply chain disruption identified")
print("  2. Industry Layer: Electronics sector most affected")
print("  3. Firm Layer: Adaptation strategies vary by impact level")
print("  4. Consumer Layer: Price increases expected across sectors")
print("  5. Geopolitical Layer: Social tension varies by sector impact")
print()

print("⚠️  RISK FACTORS:")
print("  • Supply chain complexity in electronics")
print("  • Limited alternative suppliers in short term")
print("  • Consumer price sensitivity")
print("  • Potential for retaliatory measures")
print()

print("💡 POLICY RECOMMENDATIONS:")
print("  • Monitor supply chain adaptation")
print("  • Support domestic electronics industry")
print("  • Consider consumer impact mitigation")
print("  • Engage in diplomatic dialogue")
print()

print("🎯 NEXT STEPS:")
print("  • Validate predictions with historical data")
print("  • Refine model with real-world feedback")
print("  • Extend analysis to other trade routes")
print("  • Implement continuous monitoring")

## 10. Export Results

Finally, let's export our analysis results for further use.

In [ ]:
# Export prediction results
import json
from datetime import datetime

# Prepare export data
export_data = {
    'analysis_metadata': {
        'timestamp': datetime.now().isoformat(),
        'model_version': config.model_version,
        'scenario': 'US-China Electronics Tariff Analysis',
        'analyst': 'TIPM Demo'
    },
    'tariff_shock': {
        'id': tariff_shock.tariff_id,
        'rate': tariff_shock.rate_change,
        'origin': tariff_shock.origin_country,
        'destination': tariff_shock.destination_country,
        'hs_codes': tariff_shock.hs_codes,
        'effective_date': tariff_shock.effective_date
    },
    'confidence_scores': prediction.confidence_scores,
    'scenario_comparison': [
        {
            'rate': result['tariff_rate'],
            'confidence': result['confidence']
        } for result in scenario_results
    ]
}

# Save to file
output_file = 'tipm_analysis_results.json'
with open(output_file, 'w') as f:
    json.dump(export_data, f, indent=2)

print(f"✅ Analysis results exported to: {output_file}")
print(f"📊 Data includes confidence scores and scenario comparisons")
print(f"🔍 File size: {len(json.dumps(export_data))} characters")

---

## Summary

This notebook demonstrated the complete TIPM workflow:

1. **Model Initialization**: Set up 6-layer TIPM architecture
2. **Data Generation**: Created synthetic training data
3. **Model Training**: Trained all layers with sample data
4. **Scenario Definition**: Created US-China electronics tariff shock
5. **Impact Prediction**: Ran prediction through all 6 layers
6. **Results Analysis**: Examined impacts at each layer
7. **Visualization**: Created comprehensive impact dashboards
8. **Scenario Analysis**: Compared multiple tariff rate scenarios
9. **Insights Generation**: Extracted key findings and recommendations
10. **Results Export**: Saved analysis for further use

The TIPM model provides a comprehensive framework for analyzing tariff impacts across multiple dimensions of the global economy, from immediate trade effects to long-term geopolitical consequences.

**Next Steps**: 
- Integrate real-world data sources
- Validate predictions against historical events
- Expand country coverage
- Implement real-time monitoring capabilities